In [2]:
print("checking if the kernel is working")


checking if the kernel is working


In [1]:
!pip uninstall -y numpy --quiet
!pip uninstall -y tensorflow --quiet
!pip install tensorflow==2.17 --quiet

from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import tensorflow as tf
print(tf.__version__, "must be 2.17.0")
import numpy as np
print(np.__version__, "must be 1.26.4")
#from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import random, os, sys
from glob import glob

import skimage

2024-12-04 01:14:12.325027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 01:14:12.603898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 01:14:12.676048: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 01:14:12.975833: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.17.0 must be 2.17.0
1.26.4 must be 1.26.4
Num GPUs Available:  1


I0000 00:00:1733274858.082400   16454 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733274858.298892   16454 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733274858.299313   16454 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [52]:
def average_wavelengths(chip, wavelength_indices):
    out_chip = chip[:,:,wavelength_indices]
    out_chip = np.nanmean(out_chip, axis=2)
    return out_chip

def convert_chip_to_different_spectra(chip, in_wavelengths, out_wavelengths):
    out_chip = []
    for out_wavelength_index in range(len(out_wavelengths)):
        out_wavelength_start = out_wavelengths[out_wavelength_index][0]
        out_wavelength_end = out_wavelengths[out_wavelength_index][1]
        
        in_wavelengths_to_grab = []
        for i in range(len(in_wavelengths)):
            in_wavelength = in_wavelengths[i]
            if in_wavelength > out_wavelength_start and in_wavelength < out_wavelength_end:
                in_wavelengths_to_grab.append(i)
        
        new_chip_at_wavelegnth = average_wavelengths(chip, in_wavelengths_to_grab)
        #print(new_chip_at_wavelegnth.shape)
        out_chip.append(new_chip_at_wavelegnth)
        
    out_chip = np.asarray(out_chip)
    out_chip = np.moveaxis(out_chip,0,2)
    #print(out_chip.shape)
    return out_chip

def convert_emit_chip_to_planetscope_chip(emit_chip_filename, planetscope_chip_filename, emit_wavelengths, planetscope_wavelengths):
    in_chip = np.load(emit_chip_filename)
    out_chip = convert_chip_to_different_spectra(in_chip, emit_wavelengths, planetscope_wavelengths)
    np.save(planetscope_chip_filename, out_chip)
    
def convert_whole_dataset(emit_chip_directory, planetscope_chip_directory, emit_wavelengths, planetscope_wavelengths, chipname_prefix = "chip_", chipname_start_index=0):
    if not os.path.exists(planetscope_chip_directory):
        os.mkdir(planetscope_chip_directory)
        
    in_chip_paths = glob(os.path.join(emit_chip_directory,'*'))
    for chip_index, in_chip_path in enumerate(in_chip_paths):
        out_filename = chipname_prefix + f"{chipname_start_index+chip_index:05}" + ".npy"
        out_path = os.path.join(planetscope_chip_directory, out_filename)
        
        convert_emit_chip_to_planetscope_chip(in_chip_path, out_path, emit_wavelengths, planetscope_wavelengths)
            
        
        
#Src: https://assets.planet.com/docs/Planet_PSScene_Imagery_Product_Spec_letter_screen.pdf Pg 10
planetscope_old_wavelengths = [(455, 515), (500, 590), (590, 670), (780,860)]
planetscope_new_wavelengths = [(431,452), (465,515), (513,549), (547,583), (600,620), (650,680), (697,713), (845,885)]
        
emit_wavelengths = [ 381.00558,  388.4092 ,  395.81583,  403.2254 ,  410.638  ,
        418.0536 ,  425.47214,  432.8927 ,  440.31726,  447.7428 ,
        455.17035,  462.59888,  470.0304 ,  477.46292,  484.89743,
        492.33292,  499.77142,  507.2099 ,  514.6504 ,  522.0909 ,
        529.5333 ,  536.9768 ,  544.42126,  551.8667 ,  559.3142 ,
        566.7616 ,  574.20905,  581.6585 ,  589.108  ,  596.55835,
        604.0098 ,  611.4622 ,  618.9146 ,  626.36804,  633.8215 ,
        641.2759 ,  648.7303 ,  656.1857 ,  663.6411 ,  671.09753,
        678.5539 ,  686.0103 ,  693.4677 ,  700.9251 ,  708.38354,
        715.84094,  723.2993 ,  730.7587 ,  738.2171 ,  745.6765 ,
        753.1359 ,  760.5963 ,  768.0557 ,  775.5161 ,  782.97754,
        790.4379 ,  797.89935,  805.36176,  812.8232 ,  820.2846 ,
        827.746  ,  835.2074 ,  842.66986,  850.1313 ,  857.5937 ,
        865.0551 ,  872.5176 ,  879.98004,  887.44147,  894.90393,
        902.3664 ,  909.82886,  917.2913 ,  924.7538 ,  932.21625,
        939.6788 ,  947.14026,  954.6027 ,  962.0643 ,  969.5268 ,
        976.9883 ,  984.4498 ,  991.9114 ,  999.37286, 1006.8344 ,
       1014.295  , 1021.7566 , 1029.2172 , 1036.6777 , 1044.1383 ,
       1051.5989 , 1059.0596 , 1066.5201 , 1073.9797 , 1081.4404 ,
       1088.9    , 1096.3597 , 1103.8184 , 1111.2781 , 1118.7368 ,
       1126.1964 , 1133.6552 , 1141.1129 , 1148.5717 , 1156.0304 ,
       1163.4882 , 1170.9459 , 1178.4037 , 1185.8616 , 1193.3184 ,
       1200.7761 , 1208.233  , 1215.6898 , 1223.1467 , 1230.6036 ,
       1238.0596 , 1245.5154 , 1252.9724 , 1260.4283 , 1267.8833 ,
       1275.3392 , 1282.7942 , 1290.2502 , 1297.7052 , 1305.1603 ,
       1312.6144 , 1320.0685 , 1327.5225 , 1334.9756 , 1342.4287 ,
       1349.8818 , 1357.3351 , 1364.7872 , 1372.2384 , 1379.6907 ,
       1387.1418 , 1394.5931 , 1402.0433 , 1409.4937 , 1416.944  ,
       1424.3933 , 1431.8427 , 1439.292  , 1446.7404 , 1454.1888 ,
       1461.6372 , 1469.0847 , 1476.5321 , 1483.9796 , 1491.4261 ,
       1498.8727 , 1506.3192 , 1513.7649 , 1521.2104 , 1528.655  ,
       1536.1007 , 1543.5454 , 1550.9891 , 1558.4329 , 1565.8766 ,
       1573.3193 , 1580.7621 , 1588.205  , 1595.6467 , 1603.0886 ,
       1610.5295 , 1617.9705 , 1625.4104 , 1632.8513 , 1640.2903 ,
       1647.7303 , 1655.1694 , 1662.6074 , 1670.0455 , 1677.4836 ,
       1684.9209 , 1692.358  , 1699.7952 , 1707.2314 , 1714.6667 ,
       1722.103  , 1729.5383 , 1736.9727 , 1744.4071 , 1751.8414 ,
       1759.2749 , 1766.7084 , 1774.1418 , 1781.5743 , 1789.007  ,
       1796.4385 , 1803.8701 , 1811.3008 , 1818.7314 , 1826.1611 ,
       1833.591  , 1841.0206 , 1848.4495 , 1855.8773 , 1863.3052 ,
       1870.733  , 1878.16   , 1885.5869 , 1893.013  , 1900.439  ,
       1907.864  , 1915.2892 , 1922.7133 , 1930.1375 , 1937.5607 ,
       1944.9839 , 1952.4071 , 1959.8295 , 1967.2518 , 1974.6732 ,
       1982.0946 , 1989.515  , 1996.9355 , 2004.355  , 2011.7745 ,
       2019.1931 , 2026.6118 , 2034.0304 , 2041.4471 , 2048.865  ,
       2056.2808 , 2063.6965 , 2071.1123 , 2078.5273 , 2085.9421 ,
       2093.3562 , 2100.769  , 2108.1821 , 2115.5942 , 2123.0063 ,
       2130.4175 , 2137.8289 , 2145.239  , 2152.6482 , 2160.0576 ,
       2167.467  , 2174.8755 , 2182.283  , 2189.6904 , 2197.097  ,
       2204.5034 , 2211.9092 , 2219.3147 , 2226.7195 , 2234.1233 ,
       2241.5269 , 2248.9297 , 2256.3328 , 2263.7346 , 2271.1365 ,
       2278.5376 , 2285.9387 , 2293.3386 , 2300.7378 , 2308.136  ,
       2315.5342 , 2322.9326 , 2330.3298 , 2337.7263 , 2345.1216 ,
       2352.517  , 2359.9126 , 2367.3071 , 2374.7007 , 2382.0935 ,
       2389.486  , 2396.878  , 2404.2695 , 2411.6604 , 2419.0513 ,
       2426.4402 , 2433.8303 , 2441.2183 , 2448.6064 , 2455.9944 ,
       2463.3816 , 2470]
    
emit_dataset_path_0 = "./reflectance/"
emit_dataset_path_1 = "./reflectance_1/"
emit_dataset_path_2 = "./reflectance_2/"
emit_dataset_path_3 = "./reflectance_3/"
emit_dataset_path_4 = "./reflectance_4/"
emit_dataset_path_5 = "./reflectance_5/"


# planetscope_old_dataset_path = "./dataset_for_cnn/train/x/"
# planetscope_new_dataset_path = "./dataset_for_cnn/train/y/"

if False: #manual toggle since this only needs to happen once
    #Granule 0:
    print("Processing granule 0")
    convert_whole_dataset(emit_dataset_path_0, "./dataset_for_cnn/train/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_")
    convert_whole_dataset(emit_dataset_path_0, "./dataset_for_cnn/train/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_")
    print("Granule 0 processed into train data")

    #Granule 1:
    print("Processing granule 1")
    convert_whole_dataset(emit_dataset_path_1, "./dataset_for_cnn/test/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_")
    convert_whole_dataset(emit_dataset_path_1, "./dataset_for_cnn/test/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_")
    print("Granule 1 processed into test data")

    # #Granule 2:
    # convert_whole_dataset(emit_dataset_path_2, "./dataset_for_cnn/valid/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_")
    # convert_whole_dataset(emit_dataset_path_2, "./dataset_for_cnn/valid/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_")

    # #Granule 3:
    # convert_whole_dataset(emit_dataset_path_3, "./dataset_for_cnn/train/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_", chipname_start_index=10000)
    # convert_whole_dataset(emit_dataset_path_3, "./dataset_for_cnn/train/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_", chipname_start_index=10000)

    # #Granule 4:
    # convert_whole_dataset(emit_dataset_path_4, "./dataset_for_cnn/train/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_", chipname_start_index=20000)
    # convert_whole_dataset(emit_dataset_path_4, "./dataset_for_cnn/train/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_", chipname_start_index=20000)

    # #Granule 5:
    # convert_whole_dataset(emit_dataset_path_5, "./dataset_for_cnn/train/x/", emit_wavelengths, planetscope_old_wavelengths, chipname_prefix = "chip_", chipname_start_index=30000)
    # convert_whole_dataset(emit_dataset_path_5, "./dataset_for_cnn/train/y/", emit_wavelengths, planetscope_new_wavelengths, chipname_prefix = "chip_", chipname_start_index=30000)


In [4]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, x_chip_path_dir, y_chip_path_dir, batch_size, shuffle=True):
        super().__init__()
        self.x_chip_path_dir = x_chip_path_dir
        self.y_chip_path_dir = y_chip_path_dir
        
        self.x_chip_path_list = glob(os.path.join(self.x_chip_path_dir,'*'))
        self.y_chip_path_list = glob(os.path.join(self.y_chip_path_dir,'*'))
        
        self.num_files = len(self.x_chip_path_list)
        self.index_path = list(range(self.num_files))+list(range(self.num_files))
        self.batch_size = batch_size
        self.shuffle = shuffle
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.index_path)
    
    def __getitem__(self, index):
        X = []
        Y = []
        for i in range(self.batch_size):
            x = np.load(self.x_chip_path_list[self.index_path[index]])
            y = np.load(self.y_chip_path_list[self.index_path[index]])
            
            X.append(x)
            Y.append(y)
        
        X = np.asarray(X).astype('float32')
        Y = np.asarray(Y).astype('float32')
       
        
       # # print(np.isnan(np.min(X)))
       #  x = np.asarray(x).astype('float32')
       #  x = np.zeros((1, 64, 64, 4))
       #  y = np.zeros((1,64,64,8))
       #  print(x)
        # X = tf.convert_to_tensor(X.tolist())
        # Y = tf.convert_to_tensor(Y.tolist())
        
        return X, Y
    
    def __len__(self):
        return self.num_files // self.batch_size


In [62]:
def generate_fcn_model(num_hidden_layers=3, first_layer_num_filters=32, first_layer_kernel_size=3, hidden_layer_num_filters=64, hidden_layer_kernel_size=3, layer_activation='sigmoid'):
    # num_hidden_layers must be > 1
    # *_num_filters must be positive integer - recommend 8, 16, 32, 64, etc
    # *_kernel_size must be odd positive integer - recommend 1, 3, 5, 7, 9, etc
    # see documentation for tf.keras for different activations
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(first_layer_num_filters, (first_layer_kernel_size, first_layer_kernel_size), activation=layer_activation, padding='same'))
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Conv2D(hidden_layer_num_filters, (hidden_layer_kernel_size, hidden_layer_kernel_size), activation=layer_activation, padding='same'))
    model.add(tf.keras.layers.Conv2D(8, (3, 3), activation='sigmoid', padding='same'))
    
    return model

#src: https://github.com/oekosheri/tensorflow_unet_scaling/blob/main/models.py
def conv_block(input, num_filters, conv_block_size=2, do_batchnorm=True, activation="relu"):
    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(input)
    if do_batchnorm == True:
        x = tf.keras.layers.BatchNormalization()(x)  # Not in the original network.
    x = tf.keras.layers.Activation(activation)(x)
    
    for i in range(conv_block_size-1):
        x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(x)
        if do_batchnorm == True:
            x = tf.keras.layers.BatchNormalization()(x)  # Not in the original network
        x = tf.keras.layers.Activation(activation)(x)
        
    return x

def encoder_block(input, num_filters, conv_block_size=2, do_batchnorm=True, activation="relu"):
    x = conv_block(input, num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
    p = tf.keras.layers.MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters, conv_block_size=2, do_batchnorm=True, activation="relu"):
    
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = tf.keras.layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
    return x

def generate_unet_model(unet_depth=3, num_filters_base=8, conv_block_size=2, do_batchnorm=True, activation="relu"):
    inputs = tf.keras.layers.Input((64,64,4)) #Assuming 64x64x4 chipsize
    
    s_list = []
    p_list = []
    d_list = []
    num_filters = num_filters_base    
    
    #encoding steps
    for i in range(unet_depth):
        if i == 0:
            s, p = encoder_block(inputs, num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
        else:
            s, p = encoder_block(p_list[i-1], num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
        s_list.append(s)
        p_list.append(p)
        num_filters *= 2
        
    #bridge
    b1 = conv_block(p_list[-1], num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
    num_filters = int(num_filters/2)
    
    #decoding steps:
    for i in range(unet_depth):
        if i == 0:
            d = decoder_block(b1, s_list[-1-i], num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
        else:
            d = decoder_block(d_list[i-1], s_list[-1-i], num_filters, conv_block_size=conv_block_size, do_batchnorm=do_batchnorm, activation=activation)
            
        d_list.append(d)
        num_filters = int(num_filters/2)
    
    #head
    output = tf.keras.layers.Conv2D(8, (3, 3), activation=activation, padding='same')(d_list[-1])
    model = tf.keras.models.Model(inputs, output, name="U-Net")
    return model
    

In [69]:
def train_model(model, batch_size=1, num_epochs=10, learning_rate=1e-3):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy', 'root_mean_squared_error'])
    #print(model.summary())
    training_generator = CustomDataGenerator("./dataset_for_cnn/train/x/", "./dataset_for_cnn/train/y/", batch_size)
    model.fit(training_generator, epochs = num_epochs)
    return model
    
def score_one_datapoint(y, y_pred):
    psnr = skimage.metrics.peak_signal_noise_ratio(np.asarray(y)[0,:,:,:], y_pred[0,:,:,:])
    ssim = skimage.metrics.structural_similarity(np.asarray(y)[0,:,:,:], y_pred[0,:,:,:], data_range = 1)
    mse = skimage.metrics.mean_squared_error(np.asarray(y)[0,:,:,:], y_pred[0,:,:,:])
    return psnr, ssim, mse
    
def test_and_score_model(model, verbose=False, num_samples=np.inf, debug=False):
    testing_generator = CustomDataGenerator("./dataset_for_cnn/test/x/", "./dataset_for_cnn/test/y/", 1)
    psnr_list = []
    ssim_list = []
    mse_list = []
    if verbose:
        print("Testing and Scoring Model...")
    for i in range(min(testing_generator.__len__(), num_samples)):
        x, y = testing_generator.__getitem__(i)
        y_pred = model.predict(x, steps=1, verbose=0)
        psnr, ssim, mse = score_one_datapoint(y, y_pred)
        psnr_list.append(psnr)
        ssim_list.append(ssim)
        mse_list.append(mse)
    
    avg_psnr = np.mean(np.asarray(psnr_list))
    avg_ssim = np.mean(np.asarray(ssim_list))
    avg_mse = np.mean(np.asarray(mse_list))
    
    if verbose:
        print("Average PSNR:", avg_psnr)
        print("Average SSIM:", avg_ssim)
        print("Average MSE:", avg_mse)
        
    if debug:
        return psnr_list, ssim_list, mse_list
    else:
        return avg_psnr, avg_ssim, avg_mse

In [71]:
## Training Parameters
#TODO: Play with these values first to see if you can get models to converge quickly
batch_size = 1              #Normally you want to push this as high as you can, multiples of 2, until you run out of VRAM
num_epochs = 10             #Increase to train more.  May need to be increased as batch size is increased to keep number of training steps equal
learning_rate = 1e-3        #Play with this to get models to converge faster

## FCN Model Parameters
#TODO: Grid-Search these, log PSNR, SSIM, MSE for each trained model in a spreadsheet:
num_hidden_layers=3         #try 0,1,2,3,4,5,...,12
first_layer_num_filters=32  #try 4,8,16,32,64,128
first_layer_kernel_size=3   #try 1,3,5,7,9 must be odd
hidden_layer_num_filters=64 #try 4,8,16,32,64,128
hidden_layer_kernel_size=3  #try 1,3,5,7,9 must be odd
layer_activation='sigmoid'  #try sigmoid and relu

## U-Net Model Parameters
#TODO: Grid-Search these, log PSNR, SSIM, MSE for each trained model in a spreadsheet:
unet_depth=3             #try 1,2,3,4,5,6,7,8
num_filters_base=8       #try 4,8,16,32
conv_block_size=2        #try 1,2,3,4,5,6,7,8
do_batchnorm=True        #try true/false
activation="relu"        #try sigmoid and relu

## Generate FCM
# model = generate_fcn_model(num_hidden_layers=num_hidden_layers, 
#                            first_layer_num_filters=first_layer_num_filters, 
#                            first_layer_kernel_size=first_layer_kernel_size, 
#                            hidden_layer_num_filters=hidden_layer_num_filters, 
#                            hidden_layer_kernel_size=hidden_layer_kernel_size,
#                            layer_activation=layer_activation)

## Generate U-Net
model = generate_unet_model(unet_depth=unet_depth, 
                            num_filters_base=num_filters_base, 
                            conv_block_size=conv_block_size, 
                            do_batchnorm=do_batchnorm, 
                            activation=activation)


## Train Model
model = train_model(model, batch_size=batch_size, num_epochs=num_epochs, learning_rate=learning_rate)


Epoch 1/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.2423 - loss: 0.0462 - root_mean_squared_error: 0.2110
Epoch 2/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.3432 - loss: 0.0087 - root_mean_squared_error: 0.0929
Epoch 3/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.4977 - loss: 0.0060 - root_mean_squared_error: 0.0771
Epoch 4/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6047 - loss: 0.0034 - root_mean_squared_error: 0.0583
Epoch 5/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6883 - loss: 0.0032 - root_mean_squared_error: 0.0562
Epoch 6/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7460 - loss: 0.0025 - root_mean_squared_error: 0.0495
Epoch 7/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7578 - loss: 0.0018 - root_mean_squared_error: 0.0428
Epoch 8/10
650/650 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7600 - loss: 0.0038 - root_mean_squared_error: 0.0609
Epoch 9/10
650/650 ━━━━━━━━━━━━━━━━━━━━

In [76]:
## Test Model
avg_psnr, avg_ssim, avg_mse = test_and_score_model(model, verbose=True, num_samples=10, debug=False)

Testing and Scoring Model...
Average PSNR: 31.62634021957396
Average SSIM: 0.8623859927359433
Average MSE: 0.0006904384368934478


In [77]:
avg_psnr

31.62634021957396

In [78]:
avg_ssim

0.8623859927359433

In [79]:
avg_mse

0.0006904384368934478

In [7]:
## Scratch ##

In [36]:
i = [0,1,2]
i[-0]

0

In [ ]:
# class CustomDataGenerator2(tf.keras.utils.Sequence):
#     def __init__(self, x_chip_path_dir, y_chip_path_dir, batch_size, shuffle=True):
#         self.x_chip_path_dir = x_chip_path_dir
#         self.y_chip_path_dir = y_chip_path_dir

#         # List all files in the directories
#         self.x_chip_path_list = glob(os.path.join(self.x_chip_path_dir, '*'))
#         self.y_chip_path_list = glob(os.path.join(self.y_chip_path_dir, '*'))

#         self.num_files = len(self.x_chip_path_list)
#         assert len(self.x_chip_path_list) == len(self.y_chip_path_list), \
#             "Mismatch between X and Y files."

#         self.index_path = np.arange(self.num_files)
#         self.batch_size = batch_size
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def on_epoch_end(self):
#         """Shuffle data at the end of each epoch."""
#         if self.shuffle:
#             np.random.shuffle(self.index_path)

#     def __len__(self):
#         """Number of batches per epoch."""
#         return int(np.floor(self.num_files / self.batch_size))

#     def __getitem__(self, index):
#         """Generate one batch of data."""
#         # Indices for this batch
#         batch_indices = self.index_path[index * self.batch_size:(index + 1) * self.batch_size]

#         # Initialize arrays
#         X, Y = [], []
#         for idx in batch_indices:
#             x = np.load(self.x_chip_path_list[idx])
#             y = np.load(self.y_chip_path_list[idx])

#             # Ensure shapes match model requirements
#             X.append(x)
#             Y.append(y)

#         # Convert lists to NumPy arrays
#         X = np.asarray(X, dtype=np.float32)  # Ensure compatible dtype
#         Y = np.asarray(Y, dtype=np.float32)
        
#         print("X shape:", X.shape)
#         print("Y shape:", Y.shape)
#         print("X dtype:", X.dtype)
#         print("Y dtype:", Y.dtype)

#         # Ensure that X and Y are 4D arrays (for Conv2D, for example)
#         # X shape should be (batch_size, height, width, channels), if not already
#         if X.ndim == 3:  # If it is 3D, add a channels dimension (e.g., grayscale image)
#             X = np.expand_dims(X, axis=-1)  # Convert shape to (batch_size, height, width, 1)
        
#         # Check for nested structures or unexpected object types inside X or Y
#         print(f"X is a numpy array: {isinstance(X, np.ndarray)}")
#         print(f"Y is a numpy array: {isinstance(Y, np.ndarray)}")

#         # Check if the arrays are consistent and fully flattened
#         try:
#             X = np.array(X, dtype=np.float32)
#             Y = np.array(Y, dtype=np.float32)

#             # Debug: check the contents of X and Y
#             print(f"X[0]: {X[0]}")
#             print(f"Y[0]: {Y[0]}")
#         except Exception as e:
#             print(f"Error while checking array structure: {e}")

#         # Try converting to TensorFlow tensors
#         try:
#             X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
#             Y_tensor = tf.convert_to_tensor(Y, dtype=tf.float32)

#             print(f"X tensor shape: {X_tensor.shape}")
#             print(f"Y tensor shape: {Y_tensor.shape}")
#         except ValueError as e:
#             print(f"Error converting to tensor: {e}")
#             print(f"X: {X}")
#             print(f"Y: {Y}")
#             raise e

#         return X_tensor, Y_tensor